In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns

In [35]:
import sys, os

<h3>Import data validation and try on a small dataset</h3>

In [38]:
# Import tensorflow
import tensorflow_data_validation as tfdv

# Get the working directory of the notebook. Then point to raw_data folder for the data 

#tfdv.generate_statistics_from_csv(data_location='/Users/evanderknaap/Documents/Projects/tfvalidate/raw_data/true_carlistings.csv')